In [1]:
def read_in_sentence(fn, start_token="<s>", end_token="</s>"):
    sents = []
    with open(fn) as text:
        sent = []
        for line in text.readlines():
            line = line.strip()
            sent.append(line)
            if line == end_token:
                sents.append(sent)
                sent = []
        
    return sents

In [2]:
source = read_in_sentence("./train-05/train-source.txt")
target = read_in_sentence("./train-05/train-target.txt")
#sanity check..
print(f"num sentences in source: {len(source)}")
print(f"num sentences in target: {len(target)}")


num sentences in source: 45171
num sentences in target: 45171


- 47254 tokens aligned one-to-one out of 48209 tokens
- 371 two to one
- 369 two to one
- 186 two to two
- 10 three to two
- 38102 unchanged

In [3]:
from typing import List
from itertools import chain, permutations
from collections import defaultdict, Counter

# Looking at this, it looks like we need to generate 
# "up to bigram: s1_t1, s1s2_t1 "
# "up to trigram_bigram: s1s2_t1t2, s1s2s3_t1t2"
class NGram:
    def __init__(self, source: List[List[str]], target: List[List[str]]):
        self.source = source
        self.target = target
        self.N = len(self.source)
        self.target_word_appearance = defaultdict(int)
        self.source_word_appearance = defaultdict(int)

    def generate_ngram(self, n_maps = [(1,1),(2,1),(2,2), (3,2)]):
        #pre computed ngram
        ngram_dict = defaultdict()
        aligns_with_perturb = []
        for ind in range(self.N):
            ssent, tsent = self.source[ind], self.target[ind]
            # FIXME: i am not sure how to align them. We are not given a map. 
            # We know that source is always longer than target
            # but that is about it. So I am just going to align all different possible maps in the sentence...
            # Hope there is enough overlap to tell the difference
            # Example, I don't know what if target_i maps to source_i-1 source_i. 
            # And I don't know how many maps are given per sentence...
            # so target_i -> source_i, target_i -> source_i source_i+1 as I implement
            target_count = set()
            source_count = set()
            for n_gram in n_maps:
                nsource, ntarget = n_gram
                
                target_sent_inplace = [tuple(tsent[i:i+ntarget]) for i in range(len(tsent)-ntarget+1)]
                source_sent_inplace = [tuple(ssent[i:i+nsource]) for i in range(len(ssent)-nsource+1)]
                if ntarget not in target_count:
                    for t in target_sent_inplace:
                        self.target_word_appearance[t] += 1
                    target_count.add(ntarget)
                if nsource not in source_count:
                    for s in source_sent_inplace:
                        self.source_word_appearance[s] += 1
                    source_count.add(nsource)

                aligns_with_perturb += [(sword, tar)
                                        for tar in target_sent_inplace for sword in source_sent_inplace]
            print(f"{ind}/{self.N}", end = "\r")
        return Counter(aligns_with_perturb) #(sword, starget)


In [ ]:
#All training data saved here...
#import pickle
#ngram_train = NGram(source, target)
#ngram_train.generate_ngram()
#with open('train-05/ngram_count.train', 'wb') as outputfile:
#    pickle.dump(ngram_count, outputfile)
#with open('train-05/ngram_count_target.train', 'wb') as out:
#    pickle.dump(ngram_train.target_word_appearance, out)
#with open('train-05/ngram_count_source.train', 'wb') as out:
#    pickle.dump(ngram_train.source_word_appearance, out)


In [44]:

class BiTextWordAlignment:
    #implemented as in IBM Model 1 and character tmat as well for unknown words
    def __init__(self, source: List[List[str]], target: List[List[str]]):
        self.source = source
        self.target = target
        self.fidelity_count, self.fidelity_target_count, self.fidelity_source_count = None, None, None
        self.wordCountbyN = defaultdict(int)
        self.fluency = self.calculate_fluency(self.target) #wordCountbyN is to turned into percentage afterwards
        self.fluency_prob = defaultdict(float)

    def load_fidelty_stats(self, fidel_count, overall_count_target, overall_count_source):
        self.fidelity_count, self.fidelity_target_count, self.fidelity_source_count = fidel_count, overall_count_target, overall_count_source

    def _get_fidelity_prob(self, sword: tuple, tword: tuple):
        return self.fidelity_count[(sword, tword)]/self.fidelity_target_count[tword]
        
    def calculate_fluency(self, corpus): #That is P(t)
        uniword = Counter(chain(*map(lambda x: tuple(x), corpus))) #counter |t| = 1
        self.wordCountbyN[1] = sum(uniword.values())
        biword  = Counter(chain(*map(lambda x: tuple(zip(x, x[1:])), corpus))) #counter for |t| = 2
        self.wordCountbyN[2] = sum(biword.values())
        return Counter({**uniword, **biword})

    def _get_fluency_prob(self, tword: tuple):
        # I precomputed fluency count already
        return self.fluency[tword]/self.wordCountbyN[len(tword)]

    def train_lexical_prob(self, lexicalP, n_maps=[(1, 1), (2, 1), (2, 2), (3, 2)]):
        """modified from instructor's code to align for different scenarios:
        https://colab.research.google.com/drive/1V6ZwTsBe2s7tAmHTkYqpENrzt4zOrnu3?usp=sharing#scrollTo=NLmGPFha8982
        """
        
        total = defaultdict(float)  # keys are source language words
        C = defaultdict(float)
        
        for ssent, tsent in zip(self.source, self.target):
            for n_gram in n_maps: #all possible mappings
                sent_totals = defaultdict(float)
                nsource, ntarget = n_gram
                target_sent_inplace = [tuple(tsent[i:i+ntarget])
                                    for i in range(len(tsent)-ntarget+1)]
                source_sent_inplace = [
                    tuple(ssent[i:i+nsource]) for i in range(len(ssent)-nsource+1)]

                for sword in source_sent_inplace:  # source_sent_inplace
                    for tword in target_sent_inplace:
                        sent_totals[sword] += lexicalP[(sword, tword)]
                for sword in source_sent_inplace:  # source_sent_inplace
                    for tword in target_sent_inplace:
                        C[(sword, tword)] += lexicalP[(sword, tword)]/sent_totals[sword]
                        total[tword] += lexicalP[(sword, tword)]/sent_totals[sword]
        for tword in self.fidelity_target_count:
            for sword in self.fidelity_source_count:
                lexicalP[(sword, tword)] = C[(sword, tword)] / total[tword]

    def align_chars(self):
        pass
class HMMChar: #for unknown words
    def __init__(self):
        self.emission = defaultdict(lambda: defaultdict(int)) #emission[sword_i][tword_i]
        self.count_char_tar = defaultdict(int) #[tword_i]
        self.all_char_count = 0
        self.all_chars = set()
        self.tmat = defaultdict(lambda: defaultdict(int)) #tmat[tword_i][tword_i+1]

    def leveinstein(self, sword:str, tword:str):
        ###target###
        #
        #src
        #
        #sum(replacement + insertions + deletions)
        m = len(sword)+1 # empty string as well
        n = len(tword)+1 # empty string as well
        dp_mat = [[0 for i in range(n)] for j in range(m)]
        #empty string vs target_substring
        for index in range(0, m):
            dp_mat[index][0] = index
        # empty string vs source_substring
        dp_mat[0] = list(range(0,n))

        for t in range(1, n) :
            for s in range(1, m):
                if sword[s-1] == tword[t-1]: #previous step
                    diff = 0
                else:
                    diff = 1
                dp_mat[s][t] = min(dp_mat[s-1][t-1] + diff , # replacement
                                    dp_mat[s][t-1] + 1, #deletion
                                    dp_mat[s-1][t] + 1 ) #insetion
            
        return dp_mat
    def decode(self, sword:str, tword:str):
        dp_mat = self.leveinstein(sword, tword)
        i,j = len(sword), len(tword)
        sedit = []
        tedit = []
        while (i != 0 and j != 0):
            prev_distance = min(dp_mat[i-1][j-1], dp_mat[i][j-1], dp_mat[i-1][j])

            if ( prev_distance == dp_mat[i][j]):  # match
                #print("M")
                i,j = i-1,j-1
                sedit.append(sword[i])
                tedit.append(tword[j])
            elif(j and i and prev_distance == dp_mat[i-1][j-1] ): #replacement, coming from diagonal
                #print("R")
                i, j = i-1, j-1
                sedit.append(sword[i])
                tedit.append(tword[j])
            elif(i and prev_distance == dp_mat[i-1][j] ): #deletion
                #print("D")
                i, j = i-1, j
                sedit.append(sword[i])
                tedit.append("-")
                
            elif(j and prev_distance == dp_mat[i][j-1]): #insertion
                #print("I")
                i, j = i, j-1
                sedit.append("-")
                tedit.append(tword[j])
        return sedit[::-1], tedit[::-1]
    def make_emission(self, src_tar_list: List[tuple]):
        for tup in src_tar_list:
            sedit, tedit = self.decode(tup[0], tup[1])
            tedit_lagged = tedit[1:]
            for schar, tchar in zip(sedit, tedit):
                self.emission[schar][tchar] += 1
                self.count_char_tar[tchar] += 1
                self.all_char_count += 1
                self.all_chars.add(tchar)
                if len(tedit_lagged) >= 1:
                    self.tmat[schar][tedit_lagged[0]]
                    tedit_lagged=tedit_lagged[1:]

    def get_emission_prob(self, src, tag):
        return self.emission[src][tag]/self.count_char_tar[tag]
    
    
    def get_transition_prob(self, tag_prev, tag_cur):
        return self.tmat[tag_prev][tag_cur]/self.count_char_tar[tag_cur]
    
    def get_char_tar_prob(self, tag):
        return self.count_char_tar[tag]/self.all_char_count

    def viterbi(self, srcword:str):
        V = defaultdict(lambda : defaultdict(float)) #[src][tar]
        B = defaultdict(lambda : defaultdict(str)) #backpointer

        for char in self.all_chars:
            V[0][char] = self.get_char_tar_prob(char) * self.get_emission_prob(srcword[0], char) #src -> tar
        for i in range(1,len(srcword)):
            for char in self.all_chars:
                pair = self.argmax(V,char,i)
                B[i][char] = pair[0]
                V[i][char] = pair[1]*self.get_emission_prob(srcword[i], char)
            #print(V[i])
        

        final_labels = self.get_best_tag(srcword, V, B)
        return final_labels,V

    def argmax(self, V,tchar,i):
        ans=-1
        best=None
        for s in self.all_chars:
            temp = V[i-1][s] + self.get_transition_prob(tchar,s) #target_i to target_i+1
            if temp > ans:
                ans = temp
                best = s
        return (best,ans)

    def get_best_tag(self, sent, V,B):
        best_ending = None
        best_max = -1

        for tag in self.all_chars:
            if V[len(sent) - 1][tag] > best_max:
                best_max = V[len(sent) - 1][tag]
                best_ending = tag
        seq = [best_ending]
        for i in reversed(range(1, len(sent))):
            prev = B[i][seq[-1]]
            #print( seq[-1])
            seq.append(prev)
        #print(len(seq), len(sent))
        return seq[::-1]



In [5]:
import pickle
btwa = BiTextWordAlignment(source, target)
fidel_count_train = pickle.load(open("train-05/ngram_count.train","rb"))
overall_count_target = pickle.load(open("train-05/ngram_count_target.train","rb"))
overall_count_source = pickle.load(open("train-05/ngram_count_source.train","rb"))
btwa.load_fidelty_stats(fidel_count_train,overall_count_target, overall_count_source)

In [50]:
#initalizing lexProb
lexicalP = defaultdict(lambda: defaultdict(float))
for tup in fidel_count_train:
    s,t = tup
    lexicalP[s][t] = btwa._get_fidelity_prob(s,t)

#take too long even for 1 iteration and rightfully so with the way I set it up
# lets just use lexicalP here from pure count here... So no EM :< 
#btwa.train_lexical_prob(lexicalP)


In [45]:
hmm = HMMChar()
#hmm.make_emission([("abc", "abc"), ("bca", "bca"), ("cba", "cba")])
#hmm.viterbi("cba")


(['c', 'b', 'a'],
 defaultdict(<function __main__.HMMChar.viterbi.<locals>.<lambda>()>,
             {0: defaultdict(float,
                          {'a': 0.0, 'c': 0.3333333333333333, 'b': 0.0}),
              1: defaultdict(float,
                          {'a': 0.0, 'c': 0.0, 'b': 0.3333333333333333}),
              2: defaultdict(float,
                          {'a': 0.3333333333333333, 'c': 0.0, 'b': 0.0})}))

Testing

In [ ]:
#For decoding, I will do the greedy cause of my bad setup
#given lexical probability, we just align them to the one with highest probability (fidelity *fluency)
#not very generalizable here...
# source s1, s1s2, s1s2s3
def predict_from_scratch(sSent, lexicalP):
    sTargetPredicted = []
    #if word is in here. let's see if s1s2 and s1s2s3 is there as well
    for sword in sSent:
        


In [47]:

sourceTest = read_in_sentence("test-05/test-source.txt")
targetTest = read_in_sentence("test-05/test-target.txt")

[['<s>',
  'Scéal',
  'Chathail',
  'Freeman',
  '-',
  'Téann',
  'mo',
  'dheartháir',
  'chun',
  'na',
  'Dúcharraige',
  '</s>'],
 ['<s>',
  'Mí',
  'Iúil',
  'a',
  'bhí',
  'ann',
  'i',
  'mbliain',
  'a',
  '1854',
  ',',
  'nuair',
  'a',
  'bhain',
  'an',
  'taisme',
  'seo',
  'dúinn',
  '.',
  '</s>'],
 ['<s>',
  'An',
  'deartháir',
  'ba',
  'sine',
  'agam',
  ',',
  'Sean',
  'Freeman',
  ',',
  'tugadh',
  'uainn',
  'go',
  'tobann',
  'é',
  ',',
  'agus',
  'a',
  'thásc',
  'ná',
  'a',
  'thuairisc',
  'ní',
  'raibh',
  'againn',
  'le',
  'fáil',
  '.',
  '</s>'],
 ['<s>',
  'Tráthnóna',
  'breá',
  'amháin',
  ',',
  'chuaigh',
  'sé',
  'a',
  "dh'iascaireacht",
  ';',
  'agus',
  ',',
  'ach',
  'oiread',
  'agus',
  'dá',
  'slogadh',
  'an',
  'talamh',
  'é',
  ',',
  'níor',
  'phill',
  'sé',
  '.',
  '</s>'],
 ['<s>',
  'Bhí',
  'sé',
  'go',
  'maith',
  'ina',
  'shláinte',
  'agus',
  'nuair',
  'a',
  'bhí',
  'muid',
  'ag',
  'ár',
  'ndinnéar',